# End-to-End Training (Prepare → Train → Export → Benchmark)

This notebook runs the full dev-only workflow:
1) Prepare a span-JSONL dataset under `/mnt/data`.
2) Train a token-classification model from the prepared JSONL.
3) Export to ONNX under `/mnt/c` (and optionally quantize).
4) Validate and benchmark ONNX latency.

Notes:
- Runtime is offline-first. Network is only needed when downloading base models or datasets.
- You must have a local base model/tokenizer directory (e.g. `/mnt/c/ai_models/detection/edge_deid/bert-ner-zh`).


In [ ]:
import os
from pathlib import Path

# AI_WAREHOUSE 3.0 cache layout (avoid $HOME/.cache)
os.environ.setdefault('HF_HOME', '/mnt/c/ai_cache/huggingface')
os.environ.setdefault('TRANSFORMERS_CACHE', os.environ['HF_HOME'])
os.environ.setdefault('TORCH_HOME', '/mnt/c/ai_cache/torch')
os.environ.setdefault('XDG_CACHE_HOME', '/mnt/c/ai_cache')
os.environ.setdefault('PIP_CACHE_DIR', '/mnt/c/ai_cache/pip')

for key in ('HF_HOME', 'TORCH_HOME', 'XDG_CACHE_HOME', 'PIP_CACHE_DIR'):
    Path(os.environ[key]).expanduser().mkdir(parents=True, exist_ok=True)


In [ ]:
# Prepare a deterministic synthetic dataset (offline by default)
!PYTHONPATH=src python scripts/prepare_dataset.py \
  --dataset synthetic \
  --language zh \
  --split train \
  --max-examples 500


In [ ]:
# Train from the prepared span-JSONL dataset
!PYTHONPATH=src python scripts/train_token_classifier.py \
  --model-dir /mnt/c/ai_models/detection/edge_deid/bert-ner-zh \
  --output-dir /mnt/data/training/runs/edge_deid/synthetic-ner-demo \
  --language zh \
  --input-jsonl /mnt/data/datasets/edge_deid/processed/synthetic/train/dataset.jsonl \
  --max-length 256 \
  --epochs 1 \
  --batch-size 8


In [ ]:
# Export the trained model to ONNX (copies tokenizer/config into output dir)
!PYTHONPATH=src python scripts/export_token_classifier_onnx.py \
  --model-dir /mnt/data/training/runs/edge_deid/synthetic-ner-demo \
  --output-dir /mnt/c/ai_models/detection/edge_deid/synthetic-ner-demo \
  --file-name model.onnx


In [ ]:
# Optional: quantize weights to INT8
!PYTHONPATH=src python scripts/quantize_onnx_model.py \
  --input /mnt/c/ai_models/detection/edge_deid/synthetic-ner-demo/model.onnx \
  --output /mnt/c/ai_models/detection/edge_deid/synthetic-ner-demo/model.int8.onnx

# Validate the exported ONNX model (optional argmax comparison vs PyTorch)
!PYTHONPATH=src python scripts/validate_onnx_token_classifier.py \
  --onnx-model /mnt/c/ai_models/detection/edge_deid/synthetic-ner-demo/model.onnx \
  --tokenizer-dir /mnt/c/ai_models/detection/edge_deid/synthetic-ner-demo \
  --pytorch-model-dir /mnt/data/training/runs/edge_deid/synthetic-ner-demo


In [ ]:
# Benchmark ONNX NER detector
!PYTHONPATH=src python scripts/benchmark_onnx_ner.py \
  --onnx-model /mnt/c/ai_models/detection/edge_deid/synthetic-ner-demo/model.onnx \
  --tokenizer-dir /mnt/c/ai_models/detection/edge_deid/synthetic-ner-demo \
  --runs 20
